## Image file 검색 

In [4]:
from PIL import Image
import numpy as np

# 이미지 데이터를 Average Hash로 변환(※1)
def AverageHash(inputimage, size = 16):
    img = Image.open(inputimage)                        # 이미지데이터를 연다.                 --- (※2)
    img = img.convert('L')                              # 그레이 스케일로 변환                 --- (※3)
    img = img.resize((size, size), Image.ANTIALIAS)     # 리사이즈                             --- (※4)
    pixel_data = img.getdata()                          # 픽셀데이터를 구한다.                 --- (※5)
    pixels = np.array(pixel_data)                       # Numpy배열로 변환                     --- (※6)
    pixels = pixels.reshape((size, size))               # 2차원배열로 변환                     --- (※7)
    avg = pixels.mean()                                 # 산술평균계산                         --- (※8)
    diff = 1 * (pixels > avg)                          # 평균이상과 이하로 값을 1과 0으로 변환 --- (※9)
    return diff

In [2]:
## 2진수로 가정, 해쉬값으로 변환 					-- (※10)

def np2hash(n):
    bhash = []
    for nl in ahash.tolist():
        s1 = [str(i) for i in nl]
        s2 = "".join(s1)
        i = int(s2, 2)                                        # 2진수를 정수로
        bhash.append("%04x" % i)
    return "".join(bhash)

In [3]:
## Average Hash를 표시

ahash = AverageHash('cat2.jpg')
print(ahash)

FileNotFoundError: [Errno 2] No such file or directory: 'cat2.jpg'

In [4]:
print(np2hash(ahash))

0000000000041ffc1ff41ff03ffc271c333e1bfe09fe09ff00ff003f003f003f


## 이미지 데이터를 검색해서 유사한 값을 갖는 이미지 찾기

In [5]:
from PIL import Image
import numpy as np
import os, re
search_dir = "./101_ObjectCategories"
cache_dir = "./cache_avhash"
#SIZE = 128                                                                   # 이미지 픽셀 분할 사이즈
#RATE = 0.25 

In [6]:
# 파일패스의 지정
if not os.path.exists(cache_dir):
    os.mkdir(cache_dir)

In [7]:
# 이미지 데이터를 Average  Hash로 변환
def average_hash(fname, size = 16):
    fname2 = fname[len(search_dir):]
    cache_file = cache_dir + "/" + fname2.replace('/', '_') + ".csv"
    if not os.path.exists(cache_file): 
        img = Image.open(fname)
        img = img.convert('L').resize((size, size), Image.ANTIALIAS)
        pixels = np.array(img.getdata()).reshape((size, size))
        avg = pixels.mean()
        px = 1 * (pixels > avg)
        np.savetxt(cache_file, px, fmt="%.0f", delimiter=",")
    else: 
        px = np.loadtxt(cache_file, delimiter=",")
    return px

In [8]:
# 간단한 해밍거리를 구한다.
def hamming_dist(a, b):
    aa = a.reshape(1, -1)
    ab = b.reshape(1, -1)
    dist = (aa != ab).sum()
    return dist

In [9]:
# 모든 디렉토리를 열거한다.
def enum_all_files(path):
    for root, dirs, files in os.walk(path):
        for f in files:
            fname = os.path.join(root, f)
            # Modify : 파일경로에 있는 \ 를 / 로 바꾸어 fname_temp 를 return 해 줌.
            fname_temp = fname.replace('\\', '/')
            if re.search(r'\.(jpg|jpeg|png)$', fname_temp):
                yield fname_temp
            # Modify END

In [10]:
# 이미지를 검색한다.
def find_image(fname, rate):
    src = average_hash(fname)
    for fname in enum_all_files(search_dir):
        dst = average_hash(fname)
        diff_r = hamming_dist(src, dst) / 256
        # print("[check] ",fname)
        if diff_r < rate:
            yield (diff_r, fname)

In [11]:
# 검색
srcfile = search_dir + "/chair/image_0016.jpg"
html = ""
sim = list(find_image(srcfile, 0.25))
sim = sorted(sim, key=lambda x:x[0])
for r, f in sim:
    print(r, ">", f)
    s = '<div style="float:left;"><h3>[차이:' + str(r) + '-' + \
        os.path.basename(f) + ']</h3>'+ \
        '<p><a href="' + f + '"><img src="' + f + '" width=400>'+ \
        '</a></p></div>'
    html += s

0.0 > ./101_ObjectCategories/chair/image_0016.jpg
0.22265625 > ./101_ObjectCategories/stop_sign/image_0019.jpg
0.2265625 > ./101_ObjectCategories/chair/image_0031.jpg
0.23046875 > ./101_ObjectCategories/airplanes/image_0129.jpg
0.234375 > ./101_ObjectCategories/umbrella/image_0009.jpg
0.23828125 > ./101_ObjectCategories/airplanes/image_0124.jpg
0.24609375 > ./101_ObjectCategories/chair/image_0001.jpg
0.24609375 > ./101_ObjectCategories/chair/image_0002.jpg
0.24609375 > ./101_ObjectCategories/dragonfly/image_0001.jpg


In [12]:
# HTML를 출력
html = """<html><body><h3>원이미지</h3><p>
<img src='{0}' width=400></p>{1}
</body></html>""".format(srcfile, html)
with open("./avhash-search-output.html", "w", encoding="utf-8") as f:    
    f.write(html)
print("ok")

ok


## TensorFlow을 이용한 이미지 학습

In [13]:
## 1. 이미지 데이터를 Python 데이터 형으로  변환
from sklearn import cross_validation     # Both of these will be replaced in version 0.20 by the module's model_selection equivalents
from PIL import Image
import os, glob
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [14]:
## 2. 분류대상 카테고리를 선택한다.
caltech_dir = "./101_ObjectCategories"
categories = ["chair","camera","butterfly","elephant","flamingo"]
nb_classes = len(categories)

In [15]:
##3.  이미지 사이즈를 지정
image_w = 64 
image_h = 64
pixels = image_w * image_h * 3

In [16]:
## 4. 이미지 데이터를 읽어 들인다.
X = []
Y = []
for idx, cat in enumerate(categories):
    label = [0 for i in range(nb_classes)]
    label[idx] = 1                                                         # 라벨을 지정한다. 
    image_dir = caltech_dir + "/" + cat                                    # 이미지  열거
    files = glob.glob(image_dir+"/*.jpg")
    for i,f in enumerate(files):
        img = Image.open(f)                                               # Numpy배열 변환 
        img = img.convert("RGB")
        img = img.resize((image_w, image_h))
        data = np.asarray(img)
        X.append(data)
        Y.append(label)
        if i % 10 == 0:
            print(i, "\n", data)
X = np.array(X)
Y = np.array(Y)

0 
 [[[255 250 251]
  [255 253 255]
  [254 253 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[254 255 255]
  [255 252 246]
  [255 254 243]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[239 245 245]
  [129 113  88]
  [156 131  91]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 ...

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
10 
 [[[207 187 188]
  [208 188 189]
  [206 186 185]
  ...
  [203 179 153]
  [201 177 151]
  [200 176 150]]

 [[208 188 187]
  [206 186 185]
  [209 190 186]
  ...
  [208 184 158]
  [204 180 154]
  [202 178 152]]

 [[204 185 181]
  [204 185 179]
  [204 185 179]
  ...
  [207 183 157]
  [203 179 153]
  [202 178 152]]

 ...

 [[233 220 212]
  [232 219 210]
  [229 217 

80 
 [[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
90 
 [[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
0 
 [[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 250]
  [253 253 253]
  [255 254 255]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...


In [17]:
##5.  학습 데이터와 테스트 데이터를 구분한다. 
X_train, X_test, y_train, y_test = \
    cross_validation.train_test_split(X, Y)
xy = (X_train, X_test, y_train, y_test)
np.save("./5obj.npy", xy)

print("ok,", len(Y))

ok, 334


## CNN을 적용

In [18]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import numpy as np

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [19]:
##7.  카테고리 지정
categories = ["chair","camera","butterfly","elephant","flamingo"]
nb_classes = len(categories)

##8.  이미지 사이즈 지정
image_w = 64 
image_h = 64

In [20]:
# Data를 로드
X_train, X_test, y_train, y_test = np.load("./5obj.npy")
# Data를 정규화
X_train = X_train.astype("float") / 256
X_test = X_test.astype("float") /256
print('X_train shape:', X_train.shape)

X_train shape: (250, 64, 64, 3)


In [21]:
#11. 모델을 구축
model = Sequential()
model.add(Convolution2D(32, 3, 3,
    border_mode='same',
    input_shape=X_train.shape[1:]))                                                        
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())                                                                                 
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))                                                               
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy',
    optimizer='rmsprop',
    metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(64, 64, 3..., padding="same")`
  """
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="same")`
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  if sys.path[0] == '':


In [22]:
##12. 모델을 훈련한다. 
model.fit(X_train, y_train, batch_size=32, nb_epoch=20)

##13. 모델을 평가한다.  
score = model.evaluate(X_test, y_test)
print('loss=', score[0])
print('accuracy=', score[1])

Instructions for updating:
Use tf.cast instead.


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/20
250/250 [==============================] - 4s 18ms/step - loss: 1.3701 - acc: 0.7208
Epoch 2/20
250/250 [==============================] - 2s 9ms/step - loss: 0.5115 - acc: 0.7920
Epoch 3/20
250/250 [==============================] - 2s 9ms/step - loss: 0.4877 - acc: 0.8000
Epoch 4/20
250/250 [==============================] - 2s 9ms/step - loss: 0.4746 - acc: 0.7968
Epoch 5/20
250/250 [==============================] - 2s 9ms/step - loss: 0.4345 - acc: 0.8136
Epoch 6/20
250/250 [==============================] - 2s 9ms/step - loss: 0.4834 - acc: 0.7976
Epoch 7/20
250/250 [==============================] - 2s 9ms/step - loss: 0.3804 - acc: 0.8256
Epoch 8/20
250/250 [==============================] - 2s 9ms/step - loss: 0.3213 - acc: 0.8672
Epoch 9/20
250/250 [==============================] - 2s 9ms/step - loss: 0.3019 - acc: 0.8816
Epoch 10/20
250/250 [==============================] - 2s 9ms/step - loss: 0.2353 - acc: 0.9096
Epoch 11/20
250/250 [===========================

## 정확도를 높이기 위한 방법

1. 절대적으로 부족한 sample을 만들어 내기 위해 기본 이미지를 회전시키거나 반전을 시킨다. 
2. 확대축소, 평균화하거나 노이즈를 넣어주거나 한다. 
3. 컨트라스트, 간마값을 변형한 이미지를 만들어 낸다. 
Image.transpose(v) 
Image.rotate(angle)
4. 잘못된 분류가 되어 있는지도 확인을 함으로서 성능을 개선할 수 있다. 
predict() 메쏘드를 이용해서 실제 예측을 수행하고 잘못된 이미지를 error directory에 보존하는 방법이다. 

In [23]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import h5py
from PIL import Image
import numpy as np
import os

In [28]:
# 모델을 평가 --- (※5)
pre = model.predict(X_test)
for i,v in enumerate(pre):
    pre_ans = v.argmax()
    ans = y_test[i].argmax()
    dat = X_test[i]
    if ans == pre_ans: continue
    print("[NG]", categories[pre_ans], "!=", categories[ans])
    print(v)
    fname = "error/" + str(i) + "-" + categories[pre_ans] + \
        "-ne-" + categories[ans] + ".PNG"
    dat *= 256
    img = Image.fromarray(np.uint8(dat))
    img.save(fname)

[NG] butterfly != flamingo
[0. 0. 1. 0. 0.]


FileNotFoundError: [Errno 2] No such file or directory: 'error/4-butterfly-ne-flamingo.PNG'

In [29]:
# 모델을 훈련 --- (※4)

hdf5_file = "./5obj-model.hdf5"
if os.path.exists(hdf5_file):
    model.load_weights(hdf5_file)
else:
    model.fit(X_train, y_train, batch_size=32, nb_epoch=20)
    model.save_weights(hdf5_file)

In [30]:
score = model.evaluate(X_test, y_test)
print('loss=', score[0])
print('accuracy=', score[1])

84/84 [==============================] - 0s 2ms/step
loss= 0.4370937820169188
accuracy= 0.9642856887408665
